<h3>Establish connection</h3>

In [1]:
%reload_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\Cermati\\AppData\\Roaming\\gcloud\\application_default_credentials.json"
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])
from google.cloud import bigquery

import pandas as pd
import numpy as np

C:\Users\Cermati\AppData\Roaming\gcloud\application_default_credentials.json


<h3>Get Data from BQ</h3>

In [2]:
%%bigquery df --project athena-179008
with province_reference as (
    select cities.name as _city, prov.name as _province 
    from data-platform-cermati.vayu.cermati_mongodb_cities cities 
    inner join data-platform-cermati.vayu.cermati_mongodb_provinces prov on prov._id=cities.province
)
	SELECT
			 format_date("%Y-%m",date_trunc(date(decisionReceivedDate), MONTH)) as Date,
			 case
			    when productType like '%Asuransi%' then 'Asuransi'
			 else productType end as productType

			    ,
			 case when city <> '' then city else 'DKI Jakarta' end as city
			 , case
			    when city is null then 'DKI JAKARTA'
			    when prov._province is not null then prov._province
			    when upper(city)='KABUPATEN KEPULUAN SERIBU' then 'DKI JAKARTA'
			    when lower(city) like '%jakarta%' then 'DKI JAKARTA'
			    when lower(city) like '%bandung%' then 'JAWA BARAT'
			    when lower(city) like '%tangerang%' then 'BANTEN'
			 #   when lower(city) like 'bogor' then 'Jawa Barat'
			 #   when lower(city) like 'bekasi' then 'Jawa Barat'
			 #   when lower(city) like 'depok' then 'Jawa Barat'
			 #else upper(city)
			 end as province
	  , case 
	  		when ageWhenApply < 25 then '1. age < 25 or blank' 
	  		when ageWhenApply >= 25 and ageWhenApply <= 45 then '2. age between 25 and 45'
	  		when ageWhenApply > 45 then '3. age > 45'
	  	end as ageGroup
	  , CASE 
		    when regexp_contains(applicant_gender, '(?i)(pria|laki)') then 'PRIA' 
		    when regexp_contains(applicant_gender, '(?i)(wanita|perempuan)') then 'WANITA'
		  #  WHEN applicant_gender IS NOT NULL THEN applicant_gender
		    WHEN applicant_gender is null and identifier_applicantIdentityNumber is not null and length(identifier_applicantIdentityNumber) = 16 and 
		        cast(substr(identifier_applicantIdentityNumber, 7, 2) as float64)  > 40 THEN 'WANITA'
		    WHEN applicant_gender is null and identifier_applicantIdentityNumber is not null and length(identifier_applicantIdentityNumber) = 16 and 
		        cast(substr(identifier_applicantIdentityNumber, 7, 2) as float64)  <= 40 THEN 'PRIA'
	    else null end as applicant_gender
	  , case
	        when regexp_contains(type_of_jobs, '(?i)(negeri|PNS|bumn|karyawan|pegawai|pegawai swasta)') then 'Pegawai'
	        when type_of_jobs = '' and productType in ('Asuransi Mobil', 'Asuransi Perjalanan') then 'Pelajar'
	        when type_of_jobs = 'Ibu Rumah Tangga' and productType <> 'Kartu Kredit' then 'Tidak Bekerja'
	        when type_of_jobs = '' then 'Pegawai'
	        when regexp_contains(type_of_jobs, '(?i)(wiraswasta|prof)') then 'Wiraswasta'
	    else 'Pegawai' end as type_of_jobs

	  , count(distinct orderId) apps
	  , count(distinct `user`) `users`
	  , sum(case 
	            when productType = 'Pinjaman Kredit Tanpa Agunan' then approvedLoanAmount 
	            when productType in ('Asuransi Mobil','Asuransi Perjalanan') then paymentPaidAmount
	   else null end) approvedLoanAmount
	FROM  athena-179008.vayu_data_mart.cermatimarketing_product_segmentation a
	left join province_reference prov on trim(lower(prov._city))=trim(lower(a.city))
	WHERE	 
	(
		( productType  = 'Pinjaman Kredit Tanpa Agunan' AND	Product  not like '%Kredivo%')
		or (productType = 'Kartu Kredit')
		or (
				productType  IN ( 'Asuransi Mobil'  , 'Asuransi Perjalanan'  )
				and (leadSource is null or not regexp_contains(leadSource, '(?i)(fazzcard|tcash|olx|blibli|shopee|tiket)'))	
		)
	)
	 AND	(Email  not like '%@cermati%' or Email is null)
	 AND	status  = 'Approved'
	 AND	DATE_DIFF(current_date, date(decisionReceivedDate), month)  <= 3
	 AND	DATE_DIFF(current_date, date(decisionReceivedDate), month)  <> 0	 
    # and     decisionReceivedDate >= '2019-10-01'
    # AND     decisionReceivedDate < '2020-01-01'
group by 1,2,3,4,5,6,7



<h3>Cast datatype as float</h3>

In [3]:
df['approvedLoanAmount']=pd.to_numeric(df['approvedLoanAmount'],errors = 'coerce')
df['apps'] = df.apps.astype(float)
df['users'] = df.users.astype(float)
# print(df.dtypes)
# print(df.head())

<h3>Group Data, Store Total Value & Check If all group total are equal</h3>

In [4]:
# Group by ageGroup
df_age = df.groupby('ageGroup',dropna=False).agg({
    'approvedLoanAmount' : 'sum',
    'apps' : 'sum',
    'users' : 'sum'
})
# Store Total Value for genderGroup
df_age_total = df_age.sum().to_frame()
df_age_total.reset_index(inplace=True)
df_age_total.rename(columns={0: 'Total'},inplace=True)
#---------------------------------------------------------------------------------
# Group by applicant_gender
df_gender = df.groupby('applicant_gender',dropna=False).agg({
    'approvedLoanAmount' : 'sum',
    'apps' : 'sum',
    'users' : 'sum'
})
# Store Total Value for genderGroup
df_gender_total = df_gender.sum().to_frame()
df_gender_total.reset_index(inplace=True)
df_gender_total.rename(columns={0: 'Total'},inplace=True)
#---------------------------------------------------------------------------------
# Group by type_of_jobs
df_job = df.groupby('type_of_jobs',dropna=False).agg({
    'approvedLoanAmount' : 'sum',
    'apps' : 'sum',
    'users' : 'sum'
})
# Store Total Value for genderGroup
df_job_total = df_job.sum().to_frame()
df_job_total.reset_index(inplace=True)
df_job_total.rename(columns={0: 'Total'},inplace=True)
#---------------------------------------------------------------------------------
# Group by province
df_province = df.groupby('province',dropna=False).agg({
    'approvedLoanAmount' : 'sum',
    'apps' : 'sum',
    'users' : 'sum'
})
# Store Total Value for genderGroup
df_province_total = df_province.sum().to_frame()
df_province_total.reset_index(inplace=True)
df_province_total.rename(columns={0: 'Total'},inplace=True)

print(df_gender_total.equals(df_gender_total))
print(df_gender_total.equals(df_job_total))
print(df_gender_total.equals(df_province_total))

True
True
True


<h2>ageGroup</h2>
<h3>
    <li>seperate null & non null values</li>
    <li>calculate proportion rate for non null values</li>
    <li>Distribute null values with proportion from nonnull values</li>
</h3>

In [5]:
# Seperate Non null & null value 
df_age_nonnull = df_age[~df_age.index.isnull()].copy()
df_age_null = df_age[df_age.index.isnull()].copy()

# Calculate Proportion For Non null value
df_age_nonnull['rate_approvedLoanAmount'] = (df_age_nonnull['approvedLoanAmount'] / df_age_nonnull['approvedLoanAmount'].sum())
df_age_nonnull['rate_apps'] = (df_age_nonnull['apps'] / df_age_nonnull['apps'].sum())
df_age_nonnull['rate_users'] = (df_age_nonnull['users'] / df_age_nonnull['users'].sum())

# Rename columns for null values
df_age_nonnull['id']=1
df_age_null['id']=1
df_age_null.rename(columns={'approvedLoanAmount': 'nan_approvedLoanAmount',
                   'apps': 'nan_apps',
                   'users': 'nan_users'},
          inplace=True)

# Reset Index
df_age_nonnull.reset_index(inplace=True)
df_age_null.reset_index(inplace=True)

if df_age_null.empty :
    result_age = df_age_nonnull
else : 
    # Distribute null values with proportion from nonnull values
    # Join nonnull valus with null values
    result_age = pd.merge(df_age_nonnull, df_age_null, how="left", on=["id", "id"])

    # Distribute null values with proportion from nonnull values
    result_age['add_approvedLoanAmount'] = result_age['rate_approvedLoanAmount']*result_age['nan_approvedLoanAmount']
    result_age['add_approvedLoanAmount'] = result_age['add_approvedLoanAmount'].round(0).astype(int)
    result_age['add_apps'] = result_age['rate_apps']*result_age['nan_apps']
    result_age['add_apps'] = result_age['add_apps'].round(0).astype(int)
    result_age['add_users'] = result_age['rate_users']*result_age['nan_users']
    result_age['add_users'] = result_age['add_users'].round(0).astype(int)
    result_age['new_approvedLoanAmount'] = result_age['add_approvedLoanAmount'] + result_age['approvedLoanAmount']
    result_age['new_apps'] = result_age['add_apps'] + result_age['apps']
    result_age['new_users'] = result_age['add_users'] + result_age['users']

    # Store total value
    total_approvedLoanAmount_age = df_age_total.iloc[0,1]
    total_apps_age = df_age_total.iloc[1,1]
    total_users_age = df_age_total.iloc[2,1]

    # Checking if the new values is matched with total
    age_list_approvedLoanAmount = []
    age_list_apps = []
    age_list_users = []

    age_index_length = len(result_age.index)-1

    temp_total_approvedLoanAmount_age = total_approvedLoanAmount_age
    temp_total_apps_age = total_apps_age
    temp_total_users_age = total_users_age

    for index, row in result_age.iterrows():
        temp_total_approvedLoanAmount_age = temp_total_approvedLoanAmount_age - row['new_approvedLoanAmount']
        temp_total_apps_age = temp_total_apps_age - row['new_apps']
        temp_total_users_age = temp_total_users_age - row['new_users']
        if index==age_index_length and temp_total_apps_age!=0 : 
            age_list_approvedLoanAmount.append(row['new_approvedLoanAmount']+temp_total_approvedLoanAmount_age)
            age_list_apps.append(row['new_apps']+temp_total_apps_age)
            age_list_users.append(row['new_users']+temp_total_users_age)
        else :
            age_list_approvedLoanAmount.append(row['new_approvedLoanAmount'])
            age_list_apps.append(row['new_apps'])
            age_list_users.append(row['new_users'])

    result_age['final_new_approvedLoanAmount']=age_list_approvedLoanAmount
    result_age['final_new_apps']=age_list_apps
    result_age['final_new_users']=age_list_users

<h2>applicant_gender</h2>
<h3>
    <li>seperate null & non null values</li>
    <li>calculate proportion rate for non null values</li>
    <li>Distribute null values with proportion from nonnull values</li>
</h3>

In [6]:
# Seperate Non null & null value 
df_gender_nonnull = df_gender[~df_gender.index.isnull()].copy()
df_gender_null = df_gender[df_gender.index.isnull()].copy()

# Calculate Proportion For Non null value
df_gender_nonnull['rate_approvedLoanAmount'] = (df_gender_nonnull['approvedLoanAmount'] / df_gender_nonnull['approvedLoanAmount'].sum())
df_gender_nonnull['rate_apps'] = (df_gender_nonnull['apps'] / df_gender_nonnull['apps'].sum())
df_gender_nonnull['rate_users'] = (df_gender_nonnull['users'] / df_gender_nonnull['users'].sum())

# Rename columns for null values
df_gender_nonnull['id']=1
df_gender_null['id']=1
df_gender_null.rename(columns={'approvedLoanAmount': 'nan_approvedLoanAmount',
                   'apps': 'nan_apps',
                   'users': 'nan_users'},
          inplace=True)

# Reset Index
df_gender_nonnull.reset_index(inplace=True)
df_gender_null.reset_index(inplace=True)

if df_gender_null.empty :
    result_gender = df_gender_nonnull
else : 
    # Distribute null values with proportion from nonnull values
    # Join nonnull valus with null values
    result_gender = pd.merge(df_gender_nonnull, df_gender_null, how="left", on=["id", "id"])

    # Distribute null values with proportion from nonnull values
    result_gender['add_approvedLoanAmount'] = result_gender['rate_approvedLoanAmount']*result_gender['nan_approvedLoanAmount']
    result_gender['add_approvedLoanAmount'] = result_gender['add_approvedLoanAmount'].round(0).astype(int)
    result_gender['add_apps'] = result_gender['rate_apps']*result_gender['nan_apps']
    result_gender['add_apps'] = result_gender['add_apps'].round(0).astype(int)
    result_gender['add_users'] = result_gender['rate_users']*result_gender['nan_users']
    result_gender['add_users'] = result_gender['add_users'].round(0).astype(int)
    result_gender['new_approvedLoanAmount'] = result_gender['add_approvedLoanAmount'] + result_gender['approvedLoanAmount']
    result_gender['new_apps'] = result_gender['add_apps'] + result_gender['apps']
    result_gender['new_users'] = result_gender['add_users'] + result_gender['users']

    # Store total value
    total_approvedLoanAmount_gender = df_gender_total.iloc[0,1]
    total_apps_gender = df_gender_total.iloc[1,1]
    total_users_gender = df_gender_total.iloc[2,1]

    # Checking if the new values is matched with total
    gender_list_approvedLoanAmount = []
    gender_list_apps = []
    gender_list_users = []

    gender_index_length = len(result_gender.index)-1

    temp_total_approvedLoanAmount_gender = total_approvedLoanAmount_gender
    temp_total_apps_gender = total_apps_gender
    temp_total_users_gender = total_users_gender

    for index, row in result_gender.iterrows():
        temp_total_approvedLoanAmount_gender = temp_total_approvedLoanAmount_gender - row['new_approvedLoanAmount']
        temp_total_apps_gender = temp_total_apps_gender - row['new_apps']
        temp_total_users_gender = temp_total_users_gender - row['new_users']
        if index==gender_index_length and temp_total_apps_gender!=0 : 
            gender_list_approvedLoanAmount.append(row['new_approvedLoanAmount']+temp_total_approvedLoanAmount_gender)
            gender_list_apps.append(row['new_apps']+temp_total_apps_gender)
            gender_list_users.append(row['new_users']+temp_total_users_gender)
        else :
            gender_list_approvedLoanAmount.append(row['new_approvedLoanAmount'])
            gender_list_apps.append(row['new_apps'])
            gender_list_users.append(row['new_users'])

#     result_gender['final_new_approvedLoanAmount']=gender_list_approvedLoanAmount
#     result_gender['final_new_apps']=gender_list_apps
#     result_gender['final_new_users']=gender_list_users

<h2>type_of_jobs</h2>
<h3>
    <li>seperate null & non null values</li>
    <li>calculate proportion rate for non null values</li>
    <li>Distribute null values with proportion from nonnull values</li>
</h3>

In [7]:
# Seperate Non null & null value 
df_job_nonnull = df_job[~df_job.index.isnull()].copy()
df_job_null = df_job[df_job.index.isnull()].copy()

# Calculate Proportion For Non null value
df_job_nonnull['rate_approvedLoanAmount'] = (df_job_nonnull['approvedLoanAmount'] / df_job_nonnull['approvedLoanAmount'].sum())
df_job_nonnull['rate_apps'] = (df_job_nonnull['apps'] / df_job_nonnull['apps'].sum())
df_job_nonnull['rate_users'] = (df_job_nonnull['users'] / df_job_nonnull['users'].sum())

# Rename columns for null values
df_job_nonnull['id']=1
df_job_null['id']=1
df_job_null.rename(columns={'approvedLoanAmount': 'nan_approvedLoanAmount',
                   'apps': 'nan_apps',
                   'users': 'nan_users'},
          inplace=True)

# Reset Index
df_job_nonnull.reset_index(inplace=True)
df_job_null.reset_index(inplace=True)

if df_job_null.empty :
    result_job = df_job_nonnull
else : 
    # Distribute null values with proportion from nonnull values
    # Join nonnull valus with null values
    result_job = pd.merge(df_job_nonnull, df_job_null, how="left", on=["id", "id"])

    # Distribute null values with proportion from nonnull values
    result_job['add_approvedLoanAmount'] = result_job['rate_approvedLoanAmount']*result_job['nan_approvedLoanAmount']
    result_job['add_approvedLoanAmount'] = result_job['add_approvedLoanAmount'].round(0).astype(int)
    result_job['add_apps'] = result_job['rate_apps']*result_job['nan_apps']
    result_job['add_apps'] = result_job['add_apps'].round(0).astype(int)
    result_job['add_users'] = result_job['rate_users']*result_job['nan_users']
    result_job['add_users'] = result_job['add_users'].round(0).astype(int)
    result_job['new_approvedLoanAmount'] = result_job['add_approvedLoanAmount'] + result_job['approvedLoanAmount']
    result_job['new_apps'] = result_job['add_apps'] + result_job['apps']
    result_job['new_users'] = result_job['add_users'] + result_job['users']

    # Store total value
    total_approvedLoanAmount_job = df_job_total.iloc[0,1]
    total_apps_job = df_job_total.iloc[1,1]
    total_users_job = df_job_total.iloc[2,1]

    # Checking if the new values is matched with total
    job_list_approvedLoanAmount = []
    job_list_apps = []
    job_list_users = []

    job_index_length = len(result_job.index)-1

    temp_total_approvedLoanAmount_job = total_approvedLoanAmount_job
    temp_total_apps_job = total_apps_job
    temp_total_users_job = total_users_job

    for index, row in result_job.iterrows():
        temp_total_approvedLoanAmount_job = temp_total_approvedLoanAmount_job - row['new_approvedLoanAmount']
        temp_total_apps_job = temp_total_apps_job - row['new_apps']
        temp_total_users_job = temp_total_users_job - row['new_users']
        if index==job_index_length and temp_total_apps_job!=0 : 
            job_list_approvedLoanAmount.append(row['new_approvedLoanAmount']+temp_total_approvedLoanAmount_job)
            job_list_apps.append(row['new_apps']+temp_total_apps_job)
            job_list_users.append(row['new_users']+temp_total_users_job)
        else :
            job_list_approvedLoanAmount.append(row['new_approvedLoanAmount'])
            job_list_apps.append(row['new_apps'])
            job_list_users.append(row['new_users'])

    result_job['final_new_approvedLoanAmount']=job_list_approvedLoanAmount
    result_job['final_new_apps']=job_list_apps
    result_job['final_new_users']=job_list_users

<h2>province</h2>
<h3>
    <li>seperate null & non null values</li>
    <li>calculate proportion rate for non null values</li>
    <li>Distribute null values with proportion from nonnull values</li>
</h3>

In [8]:
# Seperate Non null & null value 
df_province_nonnull = df_province[~df_province.index.isnull()].copy()
df_province_null = df_province[df_province.index.isnull()].copy()

# Calculate Proportion For Non null value
df_province_nonnull['rate_approvedLoanAmount'] = (df_province_nonnull['approvedLoanAmount'] / df_province_nonnull['approvedLoanAmount'].sum())
df_province_nonnull['rate_apps'] = (df_province_nonnull['apps'] / df_province_nonnull['apps'].sum())
df_province_nonnull['rate_users'] = (df_province_nonnull['users'] / df_province_nonnull['users'].sum())

# Rename columns for null values
df_province_nonnull['id']=1
df_province_null['id']=1
df_province_null.rename(columns={'approvedLoanAmount': 'nan_approvedLoanAmount',
                   'apps': 'nan_apps',
                   'users': 'nan_users'},
          inplace=True)

# Reset Index
df_province_nonnull.reset_index(inplace=True)
df_province_null.reset_index(inplace=True)

if df_province_null.empty :
    result_province = df_province_nonnull
else : 
    # Distribute null values with proportion from nonnull values
    # Join nonnull valus with null values
    result_province = pd.merge(df_province_nonnull, df_province_null, how="left", on=["id", "id"])

    # Distribute null values with proportion from nonnull values
    result_province['add_approvedLoanAmount'] = result_province['rate_approvedLoanAmount']*result_province['nan_approvedLoanAmount']
    result_province['add_approvedLoanAmount'] = result_province['add_approvedLoanAmount'].round(0).astype(int)
    result_province['add_apps'] = result_province['rate_apps']*result_province['nan_apps']
    result_province['add_apps'] = result_province['add_apps'].round(0).astype(int)
    result_province['add_users'] = result_province['rate_users']*result_province['nan_users']
    result_province['add_users'] = result_province['add_users'].round(0).astype(int)
    result_province['new_approvedLoanAmount'] = result_province['add_approvedLoanAmount'] + result_province['approvedLoanAmount']
    result_province['new_apps'] = result_province['add_apps'] + result_province['apps']
    result_province['new_users'] = result_province['add_users'] + result_province['users']

    # Store total value
    total_approvedLoanAmount_province = df_province_total.iloc[0,1]
    total_apps_province = df_province_total.iloc[1,1]
    total_users_province = df_province_total.iloc[2,1]

    # Checking if the new values is matched with total
    province_list_approvedLoanAmount = []
    province_list_apps = []
    province_list_users = []

    province_index_length = len(result_province.index)-1

    temp_total_approvedLoanAmount_province = total_approvedLoanAmount_province
    temp_total_apps_province = total_apps_province
    temp_total_users_province = total_users_province

    for index, row in result_province.iterrows():
        temp_total_approvedLoanAmount_province = temp_total_approvedLoanAmount_province - row['new_approvedLoanAmount']
        temp_total_apps_province = temp_total_apps_province - row['new_apps']
        temp_total_users_province = temp_total_users_province - row['new_users']
        if index==province_index_length and temp_total_apps_province!=0 : 
            province_list_approvedLoanAmount.append(row['new_approvedLoanAmount']+temp_total_approvedLoanAmount_province)
            province_list_apps.append(row['new_apps']+temp_total_apps_province)
            province_list_users.append(row['new_users']+temp_total_users_province)
        else :
            province_list_approvedLoanAmount.append(row['new_approvedLoanAmount'])
            province_list_apps.append(row['new_apps'])
            province_list_users.append(row['new_users'])

    result_province['final_new_approvedLoanAmount']=province_list_approvedLoanAmount
    result_province['final_new_apps']=province_list_apps
    result_province['final_new_users']=province_list_users

<h2>Export multiple dataframe into excel</h2>

In [9]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('OJK_Cermati.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
result_age.to_excel(writer, sheet_name='Age')
result_gender.to_excel(writer, sheet_name='Gender')
result_job.to_excel(writer, sheet_name='JobType')
result_province.to_excel(writer, sheet_name='Province')

# Close the Pandas Excel writer and output the Excel file.
writer.save()